In [6]:
"""
Used libraries for the entire project
"""
import torch
import os
import optuna
import time
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Subset, random_split, ConcatDataset, DataLoader
from PIL import Image
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve

import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam

import cv2

In [3]:
"""
-- Normalize pixel values and create training and testing datasets --

This cell normalizes the pixel values and creates a training
and testing datasets considering the stratify technique.
"""

parent_folder_path = 'images/cropped_dataset/'
categories = {'COVID': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Viral_Pneumonia': 3}
arrays = []
category_amount = []

# get the category with the least images
for category in categories.keys():
    folder_path = os.path.join(parent_folder_path, category)
    image_files = os.listdir(folder_path)
    category_amount.append(len(image_files))

max_training = min(category_amount)

# convert the images into a pytorch dataset
for cat_folder, value in categories.items():

    folder_path = os.path.join(parent_folder_path, cat_folder)
    image_files = os.listdir(folder_path)

    for i, file_name in enumerate(image_files):

        if i >= max_training: break
        file_path = os.path.join(folder_path, file_name)
        image = Image.open(file_path)
        image_array = np.array(image)

        # verify all images are of the desired size
        if image.size != (250, 250):
            print(file_path, " IS NOT 250x250, it is: ", image.size)
            continue

        if image_array.shape != (250, 250):
            image_array = np.dot(image_array[..., :3], [0.2989, 0.5870, 0.1140])

        arrays.append(image_array)

# reshape the array
arrays = arrays/ np.max(arrays)
#image_data = np.stack(arrays, axis=0)
#image_data = image_data.reshape(len(image_data), 1, 250, 250)

arrays_labels = [0] * max_training
arrays_labels += [1] * max_training
arrays_labels += [2] * max_training
arrays_labels += [3] * max_training

arrays_labels = np.array(arrays_labels)

X_train, X_test, y_train, y_test = train_test_split(arrays, arrays_labels, test_size=0.2, random_state=42, stratify=arrays_labels)

print("Finishesd")

y_train = to_categorical(y_train.astype(int), num_classes=4)
y_test = to_categorical(y_test.astype(int), num_classes=4)

Finishesd


In [5]:
# Modelo de red convolucional

model_cnn = Sequential()
model_cnn.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(250, 250, 1)))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Flatten())
model_cnn.add(Dense(64, activation='relu'))
model_cnn.add(Dense(4, activation='softmax'))

# compile the model
optimizer = Adam(learning_rate=0.01)
model_cnn.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Trian the model
model_cnn.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1)

Epoch 1/10
72/72 [==============================] - 79s 1s/step - loss: 3.3637 - accuracy: 0.4550
Epoch 2/10
72/72 [==============================] - 72s 994ms/step - loss: 0.8578 - accuracy: 0.6374
Epoch 3/10
72/72 [==============================] - 69s 962ms/step - loss: 0.7452 - accuracy: 0.7001
Epoch 4/10
72/72 [==============================] - 69s 959ms/step - loss: 0.7081 - accuracy: 0.7158
Epoch 5/10
72/72 [==============================] - 69s 965ms/step - loss: 0.6270 - accuracy: 0.7567
Epoch 6/10
72/72 [==============================] - 70s 975ms/step - loss: 0.5878 - accuracy: 0.7676
Epoch 7/10
72/72 [==============================] - 71s 988ms/step - loss: 0.5648 - accuracy: 0.7779
Epoch 8/10
72/72 [==============================] - 72s 1s/step - loss: 0.5338 - accuracy: 0.7930
Epoch 9/10
72/72 [==============================] - 73s 1s/step - loss: 0.4959 - accuracy: 0.8086
Epoch 10/10
72/72 [==============================] - 72s 997ms/step - loss: 0.4512 - accuracy: 0.832

In [9]:
pred = model_cnn.predict(X_test)
print(pred)

72/72 [==============================] - 6s 78ms/step
[[1.0206821e-01 8.4898204e-01 4.6368141e-02 2.5816644e-03]
 [2.0091405e-02 4.5901218e-01 4.2823654e-01 9.2659913e-02]
 [3.4709400e-03 7.7527940e-02 9.1899747e-01 3.5843686e-06]
 ...
 [2.2760148e-03 1.2052284e-02 9.6682256e-01 1.8849058e-02]
 [3.4139088e-01 2.0502605e-02 7.6577114e-03 6.3044888e-01]
 [6.2732227e-02 7.7702396e-02 8.5953486e-01 3.0483776e-05]]
